In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.api as sm

In [2]:
stats = pd.read_table('./regression.csv')
point = stats.iloc[:,4] / 38
rating = stats.iloc[:,5]
positional_rating = stats.iloc[:,[0,1,2,3]]

In [4]:
print(stats)

      GK    DF    MF    FW  Points  Ratings
0   6.68  7.10  7.61  7.63   100.0     7.15
1   6.83  7.20  7.17  6.97    81.0     6.98
2   6.74  7.08  6.98  7.33    77.0     6.95
3   6.70  7.01  7.01  7.45    75.0     6.99
4   6.63  6.97  7.05  7.21    70.0     6.94
5   6.75  7.10  7.18  7.10    63.0     6.92
6   6.93  7.05  6.72  6.65    54.0     6.78
7   6.63  6.85  6.64  6.81    49.0     6.67
8   6.61  6.77  6.93  6.95    47.0     6.74
9   6.75  6.91  6.75  6.73    44.0     6.72
10  6.70  6.87  6.91  7.13    44.0     6.84
11  6.52  6.77  6.69  6.71    44.0     6.65
12  6.53  6.76  6.82  6.95    42.0     6.72
13  6.54  6.78  6.76  6.63    41.0     6.66
14  6.74  6.87  6.78  6.67    40.0     6.69
15  6.57  6.74  6.69  6.55    37.0     6.61
16  6.52  6.69  6.80  6.58    36.0     6.67
17  6.89  6.71  6.53  6.63    33.0     6.59
18  6.76  6.85  6.73  6.77    33.0     6.70
19  6.50  6.80  6.68  6.66    31.0     6.63


In [5]:
# 在使用 StatsModels 拟合模型时，首先要用 add_constant 函数在每个输入数据的后面添加一个 1，借此把常数项纳入模型之中；
rating_add = sm.add_constant(rating)
# 接下来就可以调用 OLS，也就是普通最小二乘法（ordinary least squares）作为拟合对象，计算线性模型的参数；
# 最后使用 fit 函数获取拟合结果。要查看拟合模型的统计特性，只需打印出模型的 summary
est_simple = sm.OLS(point,rating_add).fit()
print(est_simple.summary())

                            OLS Regression Results                            
Dep. Variable:                 Points   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.900
Method:                 Least Squares   F-statistic:                     172.2
Date:                Sun, 10 Mar 2019   Prob (F-statistic):           1.18e-10
Time:                        18:52:53   Log-Likelihood:                 9.3960
No. Observations:                  20   AIC:                            -14.79
Df Residuals:                      18   BIC:                            -12.80
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -19.4345      1.586    -12.256      0.0

coef 表示的是参数的估计值，也就是通过最小二乘计算出的权重系数。
拟合结果 y = 3.0685x - 19.4345，这说明如果所有球员共同努力将平均评分拉高 0.1 的话，球队在每场比赛中就能平均多得 0.306 分。

std err 表示的是参数估计的标准误（standard error），虽然最小二乘得到的是无偏估计量，意味着估计结果中不存在系统误差，但每一个特定的估计值结果依然会在真实值的附近波动，标准误度量的就是估计值偏离真实值的平均程度。

最后两列 [0.025 0.975] 给出了 95% 置信区间：每个参数真实值落在这个区间内的可能性是 95%。对于线性回归而言，置信下界和上界分别是估计值减去和加上二倍的标准误。

置信区间告诉我们，平均评分拉高 0.1 并不意味着球队每场一定能多得 0.306 分，但多得的分数基本在 0.258 到 0.356 之间。如果用 2016-17 赛季的数据作为训练数据的话，这个数据的计算结果就变成了 0.33——也落在置信区间之内，这也验证的估计结果的波动性。

中间两列中的 t 和 P>|t|都是统计学中的关键指标，它们评估的是拟合结果的统计学意义。t 代表 $t$ 统计量（$t$-statistic），表示了参数的估计值和原始假设值之间的偏离程度。在线性回归中通常会假设待拟合的参数值为 0，此时的 $t$ 统计量就等于估计值除以标准误。**当数据中的噪声满足正态分布时，$t$ 统计量就满足 $t$ 分布，其绝对值越大意味着参数等于 0 的可能性越小，拟合的结果也就越可信**。

P>|t|表示的则是统计学中争议最大的指标——$p$ 值。$p$ 值（$p$-value）是在当原假设为真时，数据等于观测值或比观测值更为极端的概率。简单地说，$p$ 值表示的是数据与一个给定模型不匹配的程度，$p$ 值越小，说明数据和原假设的模型越不匹配，也就和计算出的模型越匹配。在这个例子里，原假设认为待估计的参数等于 0，而接近于 0 的 $p$ 值就意味着计算出的参数值得信任。

看完第二排再来看第一排，也就是对模型拟合数据的程度的评价，重要的指标在右侧一列。R-squared 表示的是 $R ^ 2$ 统计量，也叫作决定系数（coefficient of determination），这个取值在 [0, 1] 之间的数量表示的是输出的变化中能被输入的变化所解释的部分所占的比例。在这个例子里，$R ^ 2 = 0.905$ 意味着回归模型能够通过 $x$ 的变化解释大约 91% 的 $y$ 的变化，这表明回归模型具有良好的准确性，回归后依然不能解释的 9% 就来源于噪声。

$R ^ 2$ 统计量具有单调递增的特性，即使在模型中再添加一些和输出无关的属性，计算出来的 $R ^ 2$ 也不会下降。Adj. R-squared 就是校正版的 $R ^ 2$ 统计量。当模型中增加的变量没有统计学意义时，多余的不相关属性会使校正决定系数下降。校正决定系数体现出的是正则化的思想，它在数值上小于未校正的 $R ^ 2$ 统计量。

In [6]:
positional_rating_add = sm.add_constant(positional_rating)
est_multi = sm.OLS(point,positional_rating_add).fit()
print(est_multi.summary())

                            OLS Regression Results                            
Dep. Variable:                 Points   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     34.57
Date:                Sun, 10 Mar 2019   Prob (F-statistic):           2.09e-07
Time:                        18:57:19   Log-Likelihood:                 9.0610
No. Observations:                  20   AIC:                            -8.122
Df Residuals:                      15   BIC:                            -3.143
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -15.5306      2.366     -6.563      0.0

在这个实例中，多元回归的属性，也就是自变量被设置为每队每个位置上出场时间较多的球员的赛季平均评分的均值，所有选中球员的出场时间都在 1000 分钟以上。

利用 OLS 模型可以得到多元回归的结果，可如果对结果加以分析，就会发现一个有趣的现象：一方面，多元模型的校正决定系数是 0.876，意味着所有位置评分共同解释了输出结果的大部分变化，这也可以从预测值与真实值的散点图上观察出来；可另一方面，只有后卫评分和前锋评分的 $p$ 值低于 0.05，似乎球队的战绩只取决于这两个位置的表现。

看起来校正决定系数和 $p$ 值给出了自相矛盾的解释，这时就需要观察另外一个重要的指标：$F$ 统计量。

F 统计量主要应用在多元回归中，它检验的原假设是所有待估计的参数都等于 0，这意味着只要有一个参数不等于 0，原假设就被推翻。$F$ 统计量越大意味着原假设成立的概率越低，理想的 $F$ 值应该在百千量级。可在上面的多元回归中，$F$ 统计量仅为 34.57，这就支持了 $p$ 值的结论：估计出的参数的统计学意义并不明显。

英超数据集在统计上的非显著性可能源自过小的样本数导致的过拟合，也可能源自不同属性之间的共线性（collinearity）。可在更广泛的意义上，它揭示的却是多元线性回归无法回避的一个本质问题：**模型虽然具有足够的精确性，却缺乏关于精确性的合理解释**。

假定数据共有 10 个属性，如果只保留 10 个属性中的 5 个用于拟合的话，肯定会有不止一个 5 元属性组能够得到彼此接近的优良性能，可对不同 5 元组的解读方式却会大相径庭。这种现象，就是统计学家莱奥·布雷曼口中的“罗生门”（Rashomon）。布雷曼用这个词来描述最优模型的多重性，以及由此造成的统计建模的艰难处境：当不同的多元线性模型性能相近，却公说公有理婆说婆有理时，到底应该如何选择？

将“罗生门”深挖一步，就是机器学习和统计学在认识论上的差异：统计学讲究的是“知其然，知其所以然”，它不仅要找出数据之间的关联性，还要挖出背后的因果性，给计算出的结果赋予令人信服的解释才是统计的核心。相比之下，机器学习只看重结果，只要模型能够对未知数据做出精确的预测，那这个模型能不能讲得清楚根本不是事儿。
